# AIVLE스쿨 4기 DX트랙 5차 미니프로젝트 
## [미션#3] 중증질환 예측 모델링

[미션] 
 * Target : 중증질환 (뇌경색, 뇌출혈, 복부손상, 심근경색)
 * 데이터 분석 결과를 바탕으로 Target에 영향을 주는 Feature 전처리 (함수 정의)
 * 머신러닝/딥러닝 모델링 후 성능 비교
 * 최적AI 모델 선정 및 저장
 * 새로운 출동 이력에 제시된 환자의 증상을 바탕으로 중증 질환 예측 함수 정의

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
# 응급 출동 데이터 불러오기
# 파일명 : 119_emergency_dispatch.csv, encoding='cp949'
# 중증 질환이 ['심근경색', '복부손상', '뇌경색', '뇌출혈']인 데이터만 추출
# 데이터 랜덤으로 섞기

data = pd.read_csv("./119_emergency_dispatch.csv", encoding="cp949" )
desease = data[data['중증질환'].isin(['심근경색', '복부손상', '뇌경색', '뇌출혈'])].copy()

# 데이터 랜덤으로 섞기

desease = desease.sample(frac=1).reset_index(drop=True)


### 1) 학습용, 평가용 데이터 준비하기

* 데이터 전처리 함수 가져오기

In [3]:
# 미션2에서 정의한 preprocessing 전처리 함수 정의 가져와서 실행하기

def preprocessing(desease):

    desease = desease.copy()
    #########################################
    # 2. 데이터 전처리 하기                  #
    #########################################
    
    # '발열' 컬럼 구하기 : 체온이 37도 이상이면 1, 아니면 0
    desease['발열'] = [ 1 if x >=37 else 0 for x in desease['체온']]

    # '고혈압' 칼럼 구하기 : 수축기 혈압이 140 이상이면 1, 아니면 0
    desease['고혈압'] = [1 if x >= 140 else 0 for x in desease['수축기 혈압']]

    # '저혈압' 칼럼 구하기 : 수축기 혈압이 90 이하이면 1, 아니면 0
    desease['저혈압'] = [1 if x <= 90 else 0 for x in desease['수축기 혈압']]
    
    #########################################
    # 3. X에 선택된 Feature값 넣기           #
    #########################################
    # X : '중증질환' 및 데이터분석에서 Target에 영향을 주지 않는 칼럼 제외한 나머지
    X = desease[['체온', '수축기 혈압', '이완기 혈압', '호흡 곤란','간헐성 경련', '설사', '기침', '출혈', '통증', '만지면 아프다', 
           '무감각', '마비', '현기증', '졸도', '말이 어눌해졌다', '시력이 흐려짐', '발열', '고혈압', '저혈압']]
    
    #########################################
    # 4. X 반환하기                       #
    #########################################

    return X

In [4]:
# target 중증질환 값을 Y에 저장
# desease 데이터 프레임을 preprocessing 함수를 활용하여 데이터 전처리하여 필요한 feature만 X에 저장

Y =  desease['중증질환']
X = preprocessing(desease)

In [5]:
# AI 모델링을 위한 학습/검증 데이터 나누기 : train_test_split
# 데이터 분할 비율: 학습데이터 7 : 검증데이터 3
# random_state = 2023
# 변수명 : train_x, test_x, train_y, test_y

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state = 2023)

### 2) 모델링

 * 활용 모델 : DecisionTreeClassifier, RandomForestClassifier, XGBClassifier, DNN
 * 성능 평가 : accuracy_score

In [77]:
## Decision Tree
## 1) 불러오기

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## 2) 선언하기

model_DTC = DecisionTreeClassifier()

## 3) 학습하기

model_DTC.fit(train_x, train_y)

## 4) 예측하기
pred_DTC = model_DTC.predict(test_x)

## 5) 평가하기
print(accuracy_score(test_y, pred_DTC))


0.8825371965544244


In [78]:
## RandomForest
## 1) 불러오기

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## 2) 선언하기

model_RFC = RandomForestClassifier()

## 3) 학습하기

model_RFC.fit(train_x, train_y)

## 4) 예측하기
pred_RFC = model_RFC.predict(test_x)

## 5) 평가하기
print(accuracy_score(test_y, pred_RFC))

0.913469068128426


In [79]:
## XGBoost
## 1) 불러오기

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

## 2) 선언하기

model_XGC = XGBClassifier()


## target값 라벨링하기 {'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3}

train_y_l = train_y.map({'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3})
test_y_l = test_y.map({'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3})

## 3) 학습하기

model_XGC.fit(train_x, train_y_l)

## 4) 예측하기
pred_XGC = model_XGC.predict(test_x)

## 5) 평가하기
print(accuracy_score(test_y_l, pred_XGC))

0.9199295223179327


In [7]:
## DNN
## 1) 불러오기

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.backend import clear_session
from sklearn.metrics import accuracy_score


# 메모리 정리
clear_session()

## 2) 선언하기

model_DNN = Sequential()
model_DNN.add(Dense(32,input_shape=(train_x.shape[1],)))
model_DNN.add(Dense(64,activation='relu'))
model_DNN.add(Dropout(0.3))
model_DNN.add(Dense(16,activation='relu'))
model_DNN.add(Dense(4,activation='softmax'))

## target값 라벨링하기 {'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3}

train_y_l = train_y.map({'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3})
test_y_l = test_y.map({'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3})

## 3) 학습하기

model_DNN.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_DNN.fit(train_x, train_y_l, epochs=30)

## 4) 예측하기
pred_DNN = model_DNN.predict(test_x)

## 5) 평가하기, np.argmax(pred_DNN, axis=1)
print(accuracy_score(test_y_l, np.argmax(pred_DNN, axis=1)))

Epoch 1/30
373/373 [==============================] - 2s 2ms/step - loss: 1.6544 - accuracy: 0.2626
Epoch 2/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3906 - accuracy: 0.2685
Epoch 3/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3900 - accuracy: 0.2708
Epoch 4/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3869 - accuracy: 0.2689
Epoch 5/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3868 - accuracy: 0.2669
Epoch 6/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3877 - accuracy: 0.2667
Epoch 7/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3861 - accuracy: 0.2680
Epoch 8/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3862 - accuracy: 0.2681
Epoch 9/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3860 - accuracy: 0.2728
Epoch 10/30
373/373 [==============================] - 1s 2ms/step - loss: 1.3841 - accuracy: 0.2691

### 3) 최적 모델 선정 및 저장

In [81]:
## 질문) 최적 모델로 선정된 것은 무엇인가?



In [82]:
## 모델 저장하기

#머신러닝 모델인 경우
import joblib
joblib.dump(model_XGC, '119_model_XGC.pkl')

#딥러닝 모델인 경우
model_DNN.save('119_model_DNN.keras')

### 4) 새로운 출동 이력 데이터에 대한 중증질환 예측하기

In [83]:
# 새로운 출동 이력 데이터 : 딕셔너리 형태
new_dispatch = {
    "ID" : [500001],
    "출동일시" :['2023-04-18'],
    "이름" : ['최**'],
    "성별" : ["여성"],
    "나이" : [80],
    "체온" : [37],
    "수축기 혈압" : [145],
    "이완기 혈압" : [100],
    "호흡 곤란":[0],
    "간헐성 경련":[1],
    "설사":[0],
    "기침":[0],
    "출혈":[0],
    "통증":[1],
    "만지면 아프다":[0],
    "무감각":[0],
    "마비":[1],
    "현기증":[0],
    "졸도":[1],
    "말이 어눌해졌다":[1],
    "시력이 흐려짐":[1],

}



In [84]:
# new_dispatch 딕셔너리를 데이터 프레임으로 변환
# 변수명 : new_data

new_data = pd.DataFrame(new_dispatch)

# new_data를 preprocessing 함수를 이용하여 데이터 전처리하기
# 변수명 : new_x

new_x = preprocessing(new_data)

In [85]:
# 모델 불러오기

# 머신러닝 모델인 경우

import joblib
model_m = joblib.load('119_model_XGC.pkl')

# 딥러닝 모델인 경우

from keras.models import load_model
model_d = load_model('./119_model_DNN.keras')

In [86]:
# 중증질환 예측하기

# 머신러닝 모델인 경우
pred_new_m = model_m.predict(new_x)
print("예측값 : ", pred_new_m)

# 딥러닝 모델인 경우
pred_new_d = np.argmax(model_d.predict(new_x))
print("예측값 : ", pred_new_d)


# 중증질환 명칭으로 표시하기

sym_list = ['뇌경색', '뇌출혈', '복부손상', '심근경색']

# 머신러닝 모델인 경우
print("예측 중증질환명 : ", sym_list[pred_new_m[0]])
# 딥러닝 모델인 경우
print("예측 중증질환명 : ", sym_list[pred_new_d])



예측값 :  [0]
1/1 [==============================] - 0s 78ms/step
예측값 :  0
예측 중증질환명 :  뇌경색
예측 중증질환명 :  뇌경색


### 5) 새로운 환자(출동 이력)에 대한 중증질환 예측 함수 정의하기

 * 1. 함수 선언하기
 * 2. 데이터 준비하기
 * 3. 중증 질환 예측하기
 * 4. 중증 질환명으로 반환하기

In [87]:
# 중증질환 예측 함수 정의하기
# 함수명 : predict_disease
# 매개변수 : new_dispatch (출동 이력 데이터, 딕셔너리 형태)
# output : 중증 질환 명칭


#########################################
# 1. 함수 선언하기                       #
#########################################

def predict_disease(new_dispatch):
    
    #########################################
    # 2. 데이터 준비하기                     #
    #########################################
    
    # 중증 질환 명칭 및 라벨링 {'뇌경색':0, '뇌출혈':1, '복부손상':2, '심근경색':3}
    # 중증 질환 리스트 정의 : 라벨링 순서대로
    sym_list = ['뇌경색', '뇌출혈', '복부손상', '심근경색']
    
    # 딕셔너리 형태의 출동 이력 데이터를 데이터 프레임으로 변환
    # 변수명 : new_data

    new_data = pd.DataFrame(new_dispatch)

    # new_data를 preprocessing 함수를 이용하여 데이터 전처리된 new_x 받아오기
    # preporcessing 함수 정의 부분이 먼저 실행되어 있어야 함
    
    new_x = preprocessing(new_data) 

    #########################################
    # 3. 중증 질환 예측하기                  #
    #########################################
      
    # 저장된 AI모델 불러오기 
    # 모델 변수명 : model_m

    model_m = joblib.load('119_model_XGC.pkl')

    # new_x를 기반으로 중증질환 예측하기
    pred_new_m = model_m.predict(new_x)

    #########################################
    # 4. 중증 질환명으로 반환하기             #
    #########################################

    # 예측된 결과를 중증질환 명칭으로 반환하기
    return sym_list[pred_new_m[0]]
    
    

In [88]:
## 확인하기
# predict_disease 함수를 이용하여, 출동 이력 데이터로 중증질환 예측하기

new_dispatch = {
    "ID" : [500001],
    "출동일시" :['2023-04-18'],
    "이름" : ['최**'],
    "성별" : ["여성"],
    "나이" : [80],
    "체온" : [37],
    "수축기 혈압" : [145],
    "이완기 혈압" : [100],
    "호흡 곤란":[0],
    "간헐성 경련":[1],
    "설사":[0],
    "기침":[0],
    "출혈":[0],
    "통증":[1],
    "만지면 아프다":[0],
    "무감각":[0],
    "마비":[1],
    "현기증":[0],
    "졸도":[1],
    "말이 어눌해졌다":[1],
    "시력이 흐려짐":[1],
}


predict_disease(new_dispatch)

'뇌경색'

## 미션#3 Clear
## 수고하셨습니다!!